In [114]:
# Imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob

import scipy.signal
import holidays

import tensorflow as tf

from pandas.tseries.offsets import MonthEnd

import seaborn as sb

import IPython

# Aux Functions for Solcast and PV data processing

def get_solcast(path):
    file = pd.read_csv(path)
    file.index = pd.to_datetime(file['PeriodEnd'])
    file = file.resample('15T').pad()
    
    return file


def get_pv(path):
    file = pd.read_csv(path)
    file.index = pd.to_datetime(file['datetime_utc'])
    file = file.resample('15T').mean()
    
    return file


def get_solcastPV(df1, df2):
    '''
    df1: PV dataframe
    df2: Solcast dataframe
    '''
    
    # Filter both dataframes for 2019 and 2020
    try:
        temp_df1 = df1['2019':'2021-04-01']
        temp_df2 = df2['2019':'2021-04-01']
        
        # Check if data is complete. If not, match the smaller indexes
        if temp_df2.shape[0] < temp_df1.shape[0]:
            last_entry = temp_df2.index
            temp_df1 = temp_df1['2019':'{}'.format(temp_df2.index[-1].tz_convert(None))]


        # Only considering 2019 and 2020 since data is complete for that period
        temp_data = pd.DataFrame({'PV': temp_df1['pv'].values}, index=temp_df1.index)
        for i in np.arange(3, len(temp_df2.columns)):
            temp_data[temp_df2.columns[i]] = temp_df2[temp_df2.columns[i]].shift(-1).values
            
        return temp_data
    except:
        temp_df1 = df1['2019':'2020']
        temp_df2 = df2['2019':'2020']
        
        # Check if data is complete. If not, match the smaller indexes
        if temp_df2.shape[0] < temp_df1.shape[0]:
            last_entry = temp_df2.index
            temp_df1 = temp_df1['2019':'{}'.format(temp_df2.index[-1].tz_convert(None))]


        # Only considering 2019 and 2020 since data is complete for that period
        temp_data = pd.DataFrame({'PV': temp_df1['pv'].values}, index=temp_df1.index)
        for i in np.arange(3, len(temp_df2.columns)):
            temp_data[temp_df2.columns[i]] = temp_df2[temp_df2.columns[i]].shift(-1).values
            
        return temp_data

# Get data and build a dictionary for preprocessing

data = {}

folders = glob.glob('C:/Users/FEEL/Jupyter/ecgomes/upacs_study/data/*')
for folder in folders:
    # Load each of the files inside the folder
    temp_pv = get_pv('{}/pv.csv'.format(folder))
    temp_solcast = get_solcast('{}/solcast.csv'.format(folder))
    
    # Join the files into a single dataframe
    temp_upac = get_solcastPV(temp_pv, temp_solcast)
    
    temp_name = folder.split('\\')[1]
    data[temp_name] = temp_upac
    
    print('{} date range: {} - {}'.format(temp_name, temp_upac.index[0], temp_upac.index[-1]))

In [126]:
# Get the data from the UK dataset

data = {}

temp_data = pd.read_csv('UK/train/pv_train_set4.csv')
temp_data.index = pd.to_datetime(temp_data['datetime'])
temp_data.drop('datetime', axis=1, inplace=True)
temp_data = temp_data.resample('15T').pad()

data['uk'] = temp_data

In [127]:
# Aux Functions for adding 2D time information

import datetime

def days_2d(df):
    '''
    Adds 2D time information for single days
    df: dataframe to add the information
    '''
    # Map the index into seconds
    timestamp_s = pd.to_datetime(df.index.values).map(datetime.datetime.timestamp)
    
    # Since we're calculating the cos and sin values from seconds, it's 60 seconds into 60 min into 24 hours per day
    day_calc = 24*60*60
    
    # Calculate the values
    dayx = np.cos((2*np.pi/day_calc) * timestamp_s)
    dayy = np.sin((2*np.pi/day_calc) * timestamp_s)
    
    return dayx, dayy
    

def years_2d(df):
    '''
    Adds 2D time representation throught a year
    df: dataframe to add the information
    '''
    # Add Year Information

    day_year = df.index.dayofyear
    year_constant = 365.2524

    yearx = np.cos((2*np.pi/year_constant) * day_year)
    yeary = np.sin((2*np.pi/year_constant) * day_year)
    
    return yearx, yeary

In [128]:
# Add the 2D time information to the data

for upac in data.keys():
    dayx, dayy = days_2d(data[upac])
    yearx, yeary = years_2d(data[upac])
    
    data[upac]['Day X'] = dayx
    data[upac]['Day Y'] = dayy
    
    data[upac]['Year X'] = yearx
    data[upac]['Year Y'] = yeary

In [129]:
# Split the data for training, validation and testing

data_train = {}
data_val = {}
data_test = {}

for upac in data.keys():
    data_train[upac] = data[upac]['2018']
    data_val[upac] = data[upac]['2019-01':'2019-03']
    data_test[upac] = data[upac]['2019-04':]

C:\Users\FEEL\AppData\Local\Temp/ipykernel_14944/4210865765.py:8: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  data_train[upac] = data[upac]['2018']


In [130]:
# Aux Function for filtering data

def filter_by_points(df, frequency='D', num_points=1440, return_dictionary=False):
    
    df_dropped = df.dropna()
    grouper = df_dropped.groupby(pd.Grouper(freq=frequency))
    
    output = 0
    if return_dictionary:
        new_dict = {}
        for i in grouper:
            if (len(i[1]) != num_points):
                pass
            else:
                new_dict[i[0]] = pd.DataFrame(i[1])
        output = new_dict
    else:
        new_df = pd.DataFrame({})
        for i in grouper:
            if (len(i[1]) != num_points):
                pass
            else:
                new_df = new_df.append(pd.DataFrame(i[1]))
        output = new_df
            
    return output

In [131]:
# Filter the data by number of points that should be present in a single day

filtered_train = {}
filtered_val = {}
filtered_test = {}

for upac in data_train.keys():
    filtered_train[upac] = filter_by_points(data_train[upac], frequency='D', num_points=1440/15)
    filtered_val[upac] = filter_by_points(data_val[upac], frequency='D', num_points=1440/15)
    filtered_test[upac] = filter_by_points(data_test[upac], frequency='D', num_points=1440/15)

In [132]:
# Select columns to use

USED_COLUMNS = ['irradiance_Wm-2', 
                'pv_power_mw', 
                'panel_temp_C', 
                'Day Y', 'Day X',
                'Year Y', 'Year X']

In [133]:
# Data Normalization
# We don't want to normalize PV so we can capture diferences more easily

# Feature range
# PV - greater than 0
# Irradiance - normalize between 0 and 1
# Panel Temp - Unchanged
# Day X, Y and Year X and Y - already between -1 and 1

MAX_GHI = 1159.860000 # max value on the training set

normalized_train = {}
normalized_val = {}
normalized_test = {}

for upac in filtered_train.keys():
    normalized_train[upac] = filtered_train[upac][USED_COLUMNS].copy(deep=True)
    normalized_val[upac] = filtered_val[upac][USED_COLUMNS].copy(deep=True)
    normalized_test[upac] = filtered_test[upac][USED_COLUMNS].copy(deep=True)
    
    normalized_train[upac]['irradiance_Wm-2'] = normalized_train[upac]['irradiance_Wm-2'] / MAX_GHI
    normalized_val[upac]['irradiance_Wm-2'] = normalized_val[upac]['irradiance_Wm-2'] / MAX_GHI
    normalized_test[upac]['irradiance_Wm-2'] = normalized_test[upac]['irradiance_Wm-2'] / MAX_GHI

In [134]:
# Aux Function DNN Specific - Data Reshaping and Windowing

def split_sequence(sequence, n_steps, n_intervals):
    X = list()
    for i in np.arange(0, len(sequence), n_intervals):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence) - 1:
            break
        # gather input and output parts of the pattern
        seq_x = sequence[i:end_ix]
        X.append(seq_x)
    return np.array(X)

def split_dataframe(df, input_width):
    current_list = []
    for i in df.columns[1:]:
        current_sequence = split_sequence(df[i], input_width, 1)
        current_list.append(current_sequence)
        
    stacked_list = np.stack(current_list, axis=-1)
    
    return stacked_list


def do_windowing(df, input_width=1, y_col='pv_power_mw'):
    '''
    Perform the windowing on the dataframe
    df: input dataframe to perform windowing
    input_width: temporal dimension size or number of timesteps
    y_col: name of column for the prediction
    '''
    
    temp_x = split_dataframe(df, input_width)
    temp_y = df[y_col].values[input_width-1:-1]
    
    return temp_x, temp_y

In [136]:
# Do windowing on the data

X_train = {}
y_train = {}

X_val = {}
y_val = {}

X_test = {}
y_test = {}

for upac in normalized_train.keys():
    trainx = normalized_train[upac].drop('pv_power_mw', axis=1)
    trainy = normalized_train[upac]['pv_power_mw']
    valx = normalized_val[upac].drop('pv_power_mw', axis=1)
    valy = normalized_val[upac]['pv_power_mw']
    testx = normalized_test[upac].drop('pv_power_mw', axis=1)
    testy = normalized_test[upac]['pv_power_mw']
    
    X_train[upac] = trainx
    X_val[upac] = valx
    X_test[upac] = testx
    
    y_train[upac] = trainy
    y_val[upac] = valy
    y_test[upac] = testy

In [137]:
# Aux Functions to compile the model and add necessary info

def compile_and_fit(model, train_x, train_y, val_x, val_y,
                    max_epochs=1000, es_patience=200, rlr_patience=100, 
                    adam_lr=0.001):
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                      patience=es_patience,
                                                      mode='min')
    
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                                     patience=rlr_patience)

    model.compile(loss=tf.keras.losses.mean_squared_error,
                  optimizer=tf.optimizers.Adam(learning_rate=adam_lr),
                  metrics=[tf.metrics.MeanAbsoluteError(name='mae')])

    history = model.fit(train_x, train_y,
                        batch_size=32,
                        epochs=max_epochs,
                        validation_data=(val_x, val_y),
                        callbacks=[early_stopping, reduce_lr])
    return history

In [138]:
# Aux Function for predicting and storing values

def do_predictions(dictionary, save_path, X, y, index):
    # Go through each model in the dictionary
    for model in dictionary.keys():
        print('Doing {}'.format(model))
        
        temp_path = '{}/{}.csv'.format(save_path, model)
        
        y_pred = dictionary[model].predict(X, verbose=1)
        y_pred = pd.DataFrame(y_pred[:, :], columns=['pv_power_mw'],
                              index=index)
        
        y_pred.to_csv(temp_path)
        
    # Also save ground-truth data at the end of the loop
    y_true = pd.DataFrame(y, columns=['pv_power_mw'],
                          index=index)
    
    temp_path_gt = '{}/gt.csv'.format(save_path)
    y_true.to_csv(temp_path_gt)
    
    
def predict_upacs(model_dictionary, upac_name, X_train, y_train, X_val, y_val, X_test, y_test):
    # Simply call the function above for each of the settings to simplify
    
    print('Doing training for {}'.format(upac_name))
    temp_path = 'results/tabnet/{}_all/train'.format(upac_name)
    do_predictions(dictionary=model_dictionary, 
                   save_path=temp_path, 
                   X=X_train[upac_name], 
                   y=y_train[upac_name],
                   index=normalized_train[upac_name].index)
    IPython.display.clear_output()
    
    print('Doing validation for {}'.format(upac_name))
    temp_path = 'results/tabnet/{}_all/val'.format(upac_name)
    do_predictions(dictionary=model_dictionary, 
                   save_path=temp_path, 
                   X=X_val[upac_name], 
                   y=y_val[upac_name],
                   index=normalized_val[upac_name].index)
    IPython.display.clear_output()
    
    print('Doing testing for {}'.format(upac_name))
    temp_path = 'results/tabnet/{}_all/test'.format(upac_name)
    do_predictions(dictionary=model_dictionary, 
                   save_path=temp_path, 
                   X=X_test[upac_name], 
                   y=y_test[upac_name],
                   index=normalized_test[upac_name].index)
    IPython.display.clear_output()

In [139]:
# Import the TabNet implementation

# From: https://github.com/titu1994/tf-TabNet
from tabnet import StackedTabNetRegressor

In [140]:
# Optuna for TabNet hyperparameter optimization

import optuna
import sklearn

import random

random.seed(69)
np.random.seed(69)
tf.random.set_seed(69)

def optuna_create_model(trial):
    # Do search for n_estimators, max_depth, reg_alpha and reg_lambda
    #sug_layers = trial.suggest_int('num_layers', 1, 4)
    sug_fdim = trial.suggest_int('feature_dim', 32, 72)
    sug_odim = trial.suggest_int('output_dim', 4, 12)
    sug_dsteps = trial.suggest_int('num_decision_steps', 1, 4)
    
    sug_model = StackedTabNetRegressor(feature_columns=None,
                                       num_layers=1, #sug_layers,
                                       num_regressors=1, 
                                       feature_dim=sug_fdim,
                                       num_features=6, 
                                       output_dim=sug_odim,
                                       num_decision_steps=sug_dsteps,
                                       num_groups=1)
    
    return sug_model


def optuna_create_training(model):
    compile_and_fit(model, X_train['uk'], y_train['uk'],
                    X_val['uk'], y_val['uk'],
                    max_epochs=50, rlr_patience=10, es_patience=20)
    
    
def optuna_create_evaluation(model):
    temp_yhat = model.predict(X_val['uk'])
    return sklearn.metrics.mean_squared_error(y_val['uk'], temp_yhat)
    
    
def objective(trial):
    # Instantiate the model
    temp_model = optuna_create_model(trial)
    
    # Train the model
    optuna_create_training(temp_model)
    
    # Evaluate model
    metrics_val = optuna_create_evaluation(temp_model)
    
    return metrics_val

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100, show_progress_bar=True)

[I 2022-08-19 00:45:59,035] A new study created in memory with name: no-name-415b9484-05b3-491c-9ec2-2837bde46851
c:\users\feel\jupyter\ecgomes\upacs_study\venv\lib\site-packages\optuna\progress_bar.py:47: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch 1/50
1086/1086 [==============================] - 2s 1ms/step - loss: 0.1215 - mae: 0.1656 - val_loss: 0.0155 - val_mae: 0.0490 - lr: 0.0010
Epoch 2/50
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0284 - mae: 0.0758 - val_loss: 0.0252 - val_mae: 0.0618 - lr: 0.0010
Epoch 3/50
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0254 - mae: 0.0689 - val_loss: 0.0145 - val_mae: 0.0483 - lr: 0.0010
Epoch 4/50
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0229 - mae: 0.0635 - val_loss: 0.0149 - val_mae: 0.0489 - lr: 0.0010
Epoch 5/50
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0224 - mae: 0.0613 - val_loss: 0.0175 - val_mae: 0.0519 - lr: 0.0010
Epoch 6/50
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0213 - mae: 0.0582 - val_loss: 0.0160 - val_mae: 0.0486 - lr: 0.0010
Epoch 7/50
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0202 - mae: 0.0556 - val_loss: 0.0133 - 

1086/1086 [==============================] - 2s 1ms/step - loss: 0.1359 - mae: 0.1617 - val_loss: 0.0150 - val_mae: 0.0550 - lr: 0.0010
Epoch 2/50
1086/1086 [==============================] - 1s 968us/step - loss: 0.0261 - mae: 0.0685 - val_loss: 0.0282 - val_mae: 0.0691 - lr: 0.0010
Epoch 3/50
1086/1086 [==============================] - 1s 982us/step - loss: 0.0240 - mae: 0.0639 - val_loss: 0.0177 - val_mae: 0.0564 - lr: 0.0010
Epoch 4/50
1086/1086 [==============================] - 1s 988us/step - loss: 0.0217 - mae: 0.0586 - val_loss: 0.0147 - val_mae: 0.0503 - lr: 0.0010
Epoch 5/50
1086/1086 [==============================] - 1s 987us/step - loss: 0.0211 - mae: 0.0565 - val_loss: 0.0244 - val_mae: 0.0659 - lr: 0.0010
Epoch 6/50
1086/1086 [==============================] - 1s 987us/step - loss: 0.0198 - mae: 0.0535 - val_loss: 0.0121 - val_mae: 0.0396 - lr: 0.0010
Epoch 7/50
1086/1086 [==============================] - 1s 988us/step - loss: 0.0199 - mae: 0.0536 - val_loss: 0.0161 -

Epoch 16/50
1086/1086 [==============================] - 4s 3ms/step - loss: 0.2202 - mae: 0.2550 - val_loss: 0.1785 - val_mae: 0.2378 - lr: 0.0010
Epoch 17/50
1086/1086 [==============================] - 4s 3ms/step - loss: 0.1879 - mae: 0.2324 - val_loss: 0.6913 - val_mae: 0.5886 - lr: 0.0010
Epoch 18/50
1086/1086 [==============================] - 4s 3ms/step - loss: 0.2512 - mae: 0.2577 - val_loss: 0.6632 - val_mae: 0.5195 - lr: 0.0010
Epoch 19/50
1086/1086 [==============================] - 4s 3ms/step - loss: 0.2294 - mae: 0.2359 - val_loss: 0.6587 - val_mae: 0.5538 - lr: 0.0010
Epoch 20/50
1086/1086 [==============================] - 4s 3ms/step - loss: 0.2412 - mae: 0.2434 - val_loss: 0.6746 - val_mae: 0.5179 - lr: 0.0010
Epoch 21/50
1086/1086 [==============================] - 4s 3ms/step - loss: 0.2396 - mae: 0.2419 - val_loss: 0.6618 - val_mae: 0.4861 - lr: 0.0010
Epoch 22/50
1086/1086 [==============================] - 4s 3ms/step - loss: 0.2381 - mae: 0.2408 - val_loss: 0.

...

1086/1086 [==============================] - 1s 999us/step - loss: 0.0128 - mae: 0.0302 - val_loss: 0.0122 - val_mae: 0.0357 - lr: 1.0000e-04
Epoch 36/50
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0127 - mae: 0.0300 - val_loss: 0.0115 - val_mae: 0.0343 - lr: 1.0000e-04
[I 2022-08-19 02:15:07,765] Trial 92 finished with value: 0.011495588527024264 and parameters: {'feature_dim': 42, 'output_dim': 9, 'num_decision_steps': 1}. Best is trial 17 with value: 0.011138078476952775.
Epoch 1/50
1086/1086 [==============================] - 3s 1ms/step - loss: 0.0932 - mae: 0.1429 - val_loss: 0.0152 - val_mae: 0.0573 - lr: 0.0010
Epoch 2/50
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0269 - mae: 0.0728 - val_loss: 0.0178 - val_mae: 0.0605 - lr: 0.0010
Epoch 3/50
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0236 - mae: 0.0656 - val_loss: 0.0130 - val_mae: 0.0450 - lr: 0.0010
Epoch 4/50
1086/1086 [==============================] - 

1086/1086 [==============================] - 1s 1ms/step - loss: 0.0170 - mae: 0.0449 - val_loss: 0.0125 - val_mae: 0.0458 - lr: 0.0010
Epoch 16/50
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0161 - mae: 0.0434 - val_loss: 0.0111 - val_mae: 0.0352 - lr: 0.0010
Epoch 17/50
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0130 - mae: 0.0308 - val_loss: 0.0139 - val_mae: 0.0393 - lr: 1.0000e-04
Epoch 18/50
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0130 - mae: 0.0309 - val_loss: 0.0160 - val_mae: 0.0423 - lr: 1.0000e-04
Epoch 19/50
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0130 - mae: 0.0308 - val_loss: 0.0131 - val_mae: 0.0370 - lr: 1.0000e-04
Epoch 20/50
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0129 - mae: 0.0305 - val_loss: 0.0117 - val_mae: 0.0345 - lr: 1.0000e-04
Epoch 21/50
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0129 - mae: 0.0307 - val_loss

Epoch 5/50
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0213 - mae: 0.0630 - val_loss: 0.0265 - val_mae: 0.0708 - lr: 0.0010
Epoch 6/50
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0204 - mae: 0.0600 - val_loss: 0.0113 - val_mae: 0.0357 - lr: 0.0010
Epoch 7/50
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0198 - mae: 0.0576 - val_loss: 0.0130 - val_mae: 0.0484 - lr: 0.0010
Epoch 8/50
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0187 - mae: 0.0544 - val_loss: 0.0123 - val_mae: 0.0493 - lr: 0.0010
Epoch 9/50
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0186 - mae: 0.0542 - val_loss: 0.0443 - val_mae: 0.1001 - lr: 0.0010
Epoch 10/50
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0176 - mae: 0.0497 - val_loss: 0.0124 - val_mae: 0.0394 - lr: 0.0010
Epoch 11/50
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0180 - mae: 0.0502 - val_loss: 0.0115 

1086/1086 [==============================] - 1s 985us/step - loss: 0.0154 - mae: 0.0417 - val_loss: 0.0123 - val_mae: 0.0372 - lr: 0.0010
Epoch 23/50
1086/1086 [==============================] - 1s 982us/step - loss: 0.0154 - mae: 0.0419 - val_loss: 0.0158 - val_mae: 0.0467 - lr: 0.0010
Epoch 24/50
1086/1086 [==============================] - 1s 972us/step - loss: 0.0154 - mae: 0.0417 - val_loss: 0.0125 - val_mae: 0.0397 - lr: 0.0010
Epoch 25/50
1086/1086 [==============================] - 1s 973us/step - loss: 0.0155 - mae: 0.0418 - val_loss: 0.0131 - val_mae: 0.0446 - lr: 0.0010
Epoch 26/50
1086/1086 [==============================] - 1s 981us/step - loss: 0.0153 - mae: 0.0410 - val_loss: 0.0114 - val_mae: 0.0390 - lr: 0.0010
Epoch 27/50
1086/1086 [==============================] - 1s 980us/step - loss: 0.0128 - mae: 0.0310 - val_loss: 0.0129 - val_mae: 0.0384 - lr: 1.0000e-04
Epoch 28/50
1086/1086 [==============================] - 1s 978us/step - loss: 0.0128 - mae: 0.0302 - val_lo

Epoch 12/50
1086/1086 [==============================] - 1s 993us/step - loss: 0.0177 - mae: 0.0497 - val_loss: 0.0216 - val_mae: 0.0571 - lr: 0.0010
Epoch 13/50
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0168 - mae: 0.0472 - val_loss: 0.0133 - val_mae: 0.0421 - lr: 0.0010
Epoch 14/50
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0172 - mae: 0.0484 - val_loss: 0.0117 - val_mae: 0.0390 - lr: 0.0010
Epoch 15/50
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0170 - mae: 0.0467 - val_loss: 0.0121 - val_mae: 0.0435 - lr: 0.0010
Epoch 16/50
1086/1086 [==============================] - 1s 990us/step - loss: 0.0162 - mae: 0.0447 - val_loss: 0.0124 - val_mae: 0.0386 - lr: 0.0010
Epoch 17/50
1086/1086 [==============================] - 1s 996us/step - loss: 0.0162 - mae: 0.0435 - val_loss: 0.0114 - val_mae: 0.0383 - lr: 0.0010
Epoch 18/50
1086/1086 [==============================] - 1s 987us/step - loss: 0.0165 - mae: 0.0446 - val_

In [141]:
# TabNet best params (all fetures)

study.best_params

{'feature_dim': 43, 'output_dim': 9, 'num_decision_steps': 1}

In [142]:
# Train 10 Models for UK - All

N_MODELS = 10
uk_models_tn = {}

for i in np.arange(N_MODELS):
    IPython.display.clear_output()
    print('Current Model: {:02d}'.format(i+1))
    
    tabnet_regressor = StackedTabNetRegressor(feature_columns=None,
                                              num_layers=1, #study.best_params['num_layers'],
                                              num_regressors=1, 
                                              feature_dim=study.best_params['feature_dim'],
                                              num_features=6, 
                                              output_dim=study.best_params['output_dim'],
                                              num_decision_steps=study.best_params['num_decision_steps'],
                                              num_groups=1)
    current_net_history = compile_and_fit(tabnet_regressor, 
                                          X_train['uk'], y_train['uk'],
                                          X_val['uk'], y_val['uk'])
    
    tabnet_regressor.save('models/tabnet/uk_all/Model {:02d}'.format(i+1), save_format='tf')
    uk_models_tn['Model {:02d}'.format(i+1)] = tabnet_regressor

Current Model: 10
Epoch 1/1000
1086/1086 [==============================] - 2s 1ms/step - loss: 0.1332 - mae: 0.1793 - val_loss: 0.0134 - val_mae: 0.0528 - lr: 0.0010
Epoch 2/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0286 - mae: 0.0801 - val_loss: 0.0247 - val_mae: 0.0690 - lr: 0.0010
Epoch 3/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0254 - mae: 0.0726 - val_loss: 0.0132 - val_mae: 0.0525 - lr: 0.0010
Epoch 4/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0234 - mae: 0.0675 - val_loss: 0.0138 - val_mae: 0.0540 - lr: 0.0010
Epoch 5/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0224 - mae: 0.0647 - val_loss: 0.0163 - val_mae: 0.0532 - lr: 0.0010
Epoch 6/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0215 - mae: 0.0618 - val_loss: 0.0138 - val_mae: 0.0472 - lr: 0.0010
Epoch 7/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0202 - m

1086/1086 [==============================] - 1s 1ms/step - loss: 0.0141 - mae: 0.0380 - val_loss: 0.0137 - val_mae: 0.0422 - lr: 0.0010
Epoch 56/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0139 - mae: 0.0379 - val_loss: 0.0231 - val_mae: 0.0566 - lr: 0.0010
Epoch 57/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0142 - mae: 0.0389 - val_loss: 0.0145 - val_mae: 0.0432 - lr: 0.0010
Epoch 58/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0138 - mae: 0.0373 - val_loss: 0.0152 - val_mae: 0.0461 - lr: 0.0010
Epoch 59/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0141 - mae: 0.0382 - val_loss: 0.0120 - val_mae: 0.0377 - lr: 0.0010
Epoch 60/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0141 - mae: 0.0385 - val_loss: 0.0127 - val_mae: 0.0423 - lr: 0.0010
Epoch 61/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0140 - mae: 0.0377 - val_loss: 0.

Epoch 110/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0123 - mae: 0.0334 - val_loss: 0.0137 - val_mae: 0.0412 - lr: 0.0010
Epoch 111/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0127 - mae: 0.0340 - val_loss: 0.0185 - val_mae: 0.0498 - lr: 0.0010
Epoch 112/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0127 - mae: 0.0347 - val_loss: 0.0143 - val_mae: 0.0426 - lr: 0.0010
Epoch 113/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0126 - mae: 0.0335 - val_loss: 0.0119 - val_mae: 0.0367 - lr: 0.0010
Epoch 114/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0125 - mae: 0.0336 - val_loss: 0.0142 - val_mae: 0.0453 - lr: 0.0010
Epoch 115/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0128 - mae: 0.0349 - val_loss: 0.0139 - val_mae: 0.0416 - lr: 0.0010
Epoch 116/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0124 - mae: 

1086/1086 [==============================] - 1s 1ms/step - loss: 0.0114 - mae: 0.0339 - val_loss: 0.0142 - val_mae: 0.0478 - lr: 0.0010
Epoch 165/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0115 - mae: 0.0336 - val_loss: 0.0151 - val_mae: 0.0430 - lr: 0.0010
Epoch 166/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0116 - mae: 0.0332 - val_loss: 0.0147 - val_mae: 0.0424 - lr: 0.0010
Epoch 167/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0114 - mae: 0.0332 - val_loss: 0.0123 - val_mae: 0.0374 - lr: 0.0010
Epoch 168/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0113 - mae: 0.0324 - val_loss: 0.0149 - val_mae: 0.0445 - lr: 0.0010
Epoch 169/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0113 - mae: 0.0330 - val_loss: 0.0156 - val_mae: 0.0452 - lr: 0.0010
Epoch 170/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0126 - mae: 0.0353 - val_lo

1086/1086 [==============================] - 1s 1ms/step - loss: 0.0080 - mae: 0.0256 - val_loss: 0.0160 - val_mae: 0.0447 - lr: 1.0000e-04
Epoch 219/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0079 - mae: 0.0254 - val_loss: 0.0161 - val_mae: 0.0443 - lr: 1.0000e-04
Epoch 220/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0079 - mae: 0.0255 - val_loss: 0.0152 - val_mae: 0.0431 - lr: 1.0000e-04
Epoch 221/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0078 - mae: 0.0256 - val_loss: 0.0155 - val_mae: 0.0431 - lr: 1.0000e-04
Epoch 222/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0078 - mae: 0.0255 - val_loss: 0.0157 - val_mae: 0.0438 - lr: 1.0000e-04
Epoch 223/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0078 - mae: 0.0256 - val_loss: 0.0155 - val_mae: 0.0432 - lr: 1.0000e-04
Epoch 224/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.007

Epoch 271/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0067 - mae: 0.0251 - val_loss: 0.0152 - val_mae: 0.0431 - lr: 1.0000e-04
Epoch 272/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0068 - mae: 0.0250 - val_loss: 0.0157 - val_mae: 0.0441 - lr: 1.0000e-04
Epoch 273/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0066 - mae: 0.0251 - val_loss: 0.0156 - val_mae: 0.0435 - lr: 1.0000e-04
Epoch 274/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0067 - mae: 0.0251 - val_loss: 0.0159 - val_mae: 0.0443 - lr: 1.0000e-04
Epoch 275/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0066 - mae: 0.0250 - val_loss: 0.0155 - val_mae: 0.0439 - lr: 1.0000e-04
Epoch 276/1000
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0066 - mae: 0.0251 - val_loss: 0.0157 - val_mae: 0.0437 - lr: 1.0000e-04
Epoch 277/1000
1086/1086 [==============================] - 1s 1ms/ste

In [143]:
# UPAC08 Predictions for all features

predict_upacs(model_dictionary=uk_models_tn, 
              upac_name='uk',
              X_train=X_train, y_train=y_train,
              X_val=X_val, y_val=y_val,
              X_test=X_test, y_test=y_test)

##################################### Ghi #####################################

In [144]:
# Optuna for TabNet hyperparameter optimization

import optuna
import sklearn

import random

random.seed(69)
np.random.seed(69)
tf.random.set_seed(69)

def optuna_create_model_ghi(trial):
    # Do search for n_estimators, max_depth, reg_alpha and reg_lambda
    #sug_layers = trial.suggest_int('num_layers', 1, 4)
    sug_fdim = trial.suggest_int('feature_dim', 32, 72)
    sug_odim = trial.suggest_int('output_dim', 4, 12)
    sug_dsteps = trial.suggest_int('num_decision_steps', 1, 4)
    
    sug_model = StackedTabNetRegressor(feature_columns=None,
                                       num_layers=1, #sug_layers,
                                       num_regressors=1, 
                                       feature_dim=sug_fdim,
                                       num_features=1, 
                                       output_dim=sug_odim,
                                       num_decision_steps=sug_dsteps,
                                       num_groups=1)
    
    return sug_model


def optuna_create_training_ghi(model):
    compile_and_fit(model, X_train['uk']['irradiance_Wm-2'].values.reshape(-1, 1), y_train['uk'],
                    X_val['uk']['irradiance_Wm-2'].values.reshape(-1, 1), y_val['uk'],
                    max_epochs=50, rlr_patience=10, es_patience=20)
    
    
def optuna_create_evaluation_ghi(model):
    temp_yhat = model.predict(X_val['uk']['irradiance_Wm-2'].values.reshape(-1, 1))
    return sklearn.metrics.mean_squared_error(y_val['uk'], temp_yhat)
    
    
def objective_ghi(trial):
    # Instantiate the model
    temp_model = optuna_create_model_ghi(trial)
    
    # Train the model
    optuna_create_training_ghi(temp_model)
    
    # Evaluate model
    metrics_val = optuna_create_evaluation_ghi(temp_model)
    
    return metrics_val

study_ghi = optuna.create_study(direction='minimize')
study_ghi.optimize(objective_ghi, n_trials=100, show_progress_bar=True)

[I 2022-08-19 03:32:23,534] A new study created in memory with name: no-name-f47ebaa1-7061-4a09-a9ad-6336b1a5ca21
c:\users\feel\jupyter\ecgomes\upacs_study\venv\lib\site-packages\optuna\progress_bar.py:47: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/100 [00:00<?, ?it/s]

[TabNet]: 28 features will be used for decision steps.
Epoch 1/50
1086/1086 [==============================] - 4s 2ms/step - loss: 0.0434 - mae: 0.0802 - val_loss: 0.0133 - val_mae: 0.0408 - lr: 0.0010
Epoch 2/50
1086/1086 [==============================] - 2s 2ms/step - loss: 0.0238 - mae: 0.0576 - val_loss: 0.0134 - val_mae: 0.0394 - lr: 0.0010
Epoch 3/50
1086/1086 [==============================] - 2s 2ms/step - loss: 0.0225 - mae: 0.0552 - val_loss: 0.0151 - val_mae: 0.0482 - lr: 0.0010
Epoch 4/50
1086/1086 [==============================] - 2s 2ms/step - loss: 0.0218 - mae: 0.0548 - val_loss: 0.0165 - val_mae: 0.0502 - lr: 0.0010
Epoch 5/50
1086/1086 [==============================] - 2s 2ms/step - loss: 0.0209 - mae: 0.0531 - val_loss: 0.0288 - val_mae: 0.0706 - lr: 0.0010
Epoch 6/50
1086/1086 [==============================] - 2s 2ms/step - loss: 0.0204 - mae: 0.0520 - val_loss: 0.0172 - val_mae: 0.0603 - lr: 0.0010
Epoch 7/50
1086/1086 [==============================] - 2s 2ms/

Epoch 10/50
1086/1086 [==============================] - 3s 2ms/step - loss: 0.0190 - mae: 0.0475 - val_loss: 0.0157 - val_mae: 0.0442 - lr: 0.0010
Epoch 11/50
1086/1086 [==============================] - 3s 2ms/step - loss: 0.0190 - mae: 0.0474 - val_loss: 0.0153 - val_mae: 0.0448 - lr: 0.0010
Epoch 12/50
1086/1086 [==============================] - 3s 2ms/step - loss: 0.0189 - mae: 0.0469 - val_loss: 0.0231 - val_mae: 0.0644 - lr: 0.0010
Epoch 13/50
1086/1086 [==============================] - 3s 2ms/step - loss: 0.0179 - mae: 0.0445 - val_loss: 0.0153 - val_mae: 0.0443 - lr: 1.0000e-04
Epoch 14/50
1086/1086 [==============================] - 3s 2ms/step - loss: 0.0178 - mae: 0.0441 - val_loss: 0.0158 - val_mae: 0.0464 - lr: 1.0000e-04
Epoch 15/50
1086/1086 [==============================] - 3s 2ms/step - loss: 0.0178 - mae: 0.0442 - val_loss: 0.0168 - val_mae: 0.0486 - lr: 1.0000e-04
Epoch 16/50
1086/1086 [==============================] - 3s 2ms/step - loss: 0.0179 - mae: 0.0443 - 

1086/1086 [==============================] - 3s 3ms/step - loss: 0.0179 - mae: 0.0446 - val_loss: 0.0163 - val_mae: 0.0484 - lr: 1.0000e-04
Epoch 18/50
1086/1086 [==============================] - 3s 3ms/step - loss: 0.0179 - mae: 0.0446 - val_loss: 0.0208 - val_mae: 0.0576 - lr: 1.0000e-04
Epoch 19/50
1086/1086 [==============================] - 3s 3ms/step - loss: 0.0179 - mae: 0.0448 - val_loss: 0.0176 - val_mae: 0.0507 - lr: 1.0000e-04
Epoch 20/50
1086/1086 [==============================] - 3s 3ms/step - loss: 0.0178 - mae: 0.0447 - val_loss: 0.0157 - val_mae: 0.0464 - lr: 1.0000e-04
Epoch 21/50
1086/1086 [==============================] - 3s 3ms/step - loss: 0.0178 - mae: 0.0446 - val_loss: 0.0162 - val_mae: 0.0489 - lr: 1.0000e-04
Epoch 22/50
1086/1086 [==============================] - 3s 3ms/step - loss: 0.0178 - mae: 0.0446 - val_loss: 0.0160 - val_mae: 0.0481 - lr: 1.0000e-04
[I 2022-08-19 03:36:48,358] Trial 4 finished with value: 0.016030643442414186 and parameters: {'feat

...

Epoch 15/50
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0178 - mae: 0.0445 - val_loss: 0.0167 - val_mae: 0.0485 - lr: 1.0000e-04
Epoch 16/50
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0179 - mae: 0.0445 - val_loss: 0.0163 - val_mae: 0.0474 - lr: 1.0000e-04
Epoch 17/50
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0179 - mae: 0.0443 - val_loss: 0.0162 - val_mae: 0.0479 - lr: 1.0000e-04
Epoch 18/50
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0179 - mae: 0.0443 - val_loss: 0.0212 - val_mae: 0.0572 - lr: 1.0000e-04
Epoch 19/50
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0179 - mae: 0.0444 - val_loss: 0.0178 - val_mae: 0.0511 - lr: 1.0000e-04
Epoch 20/50
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0178 - mae: 0.0443 - val_loss: 0.0157 - val_mae: 0.0466 - lr: 1.0000e-04
Epoch 21/50
1086/1086 [==============================] - 1s 1ms/step - loss: 0.0178 - ma

1086/1086 [==============================] - 1s 965us/step - loss: 0.0178 - mae: 0.0442 - val_loss: 0.0165 - val_mae: 0.0487 - lr: 1.0000e-04
Epoch 22/50
1086/1086 [==============================] - 1s 956us/step - loss: 0.0179 - mae: 0.0443 - val_loss: 0.0161 - val_mae: 0.0488 - lr: 1.0000e-04
[I 2022-08-19 04:49:45,863] Trial 92 finished with value: 0.016133042824662172 and parameters: {'feature_dim': 37, 'output_dim': 11, 'num_decision_steps': 1}. Best is trial 89 with value: 0.015431425487623074.
[TabNet]: 26 features will be used for decision steps.
Epoch 1/50
1086/1086 [==============================] - 4s 2ms/step - loss: 0.0590 - mae: 0.0942 - val_loss: 0.0139 - val_mae: 0.0443 - lr: 0.0010
Epoch 2/50
1086/1086 [==============================] - 2s 2ms/step - loss: 0.0251 - mae: 0.0621 - val_loss: 0.0134 - val_mae: 0.0457 - lr: 0.0010
Epoch 3/50
1086/1086 [==============================] - 2s 2ms/step - loss: 0.0231 - mae: 0.0571 - val_loss: 0.0148 - val_mae: 0.0458 - lr: 0.001

1086/1086 [==============================] - 2s 2ms/step - loss: 0.0179 - mae: 0.0444 - val_loss: 0.0161 - val_mae: 0.0489 - lr: 1.0000e-04
[I 2022-08-19 04:51:45,536] Trial 94 finished with value: 0.016092846082543317 and parameters: {'feature_dim': 35, 'output_dim': 11, 'num_decision_steps': 3}. Best is trial 89 with value: 0.015431425487623074.
[TabNet]: 29 features will be used for decision steps.
Epoch 1/50
1086/1086 [==============================] - 4s 3ms/step - loss: 0.0386 - mae: 0.0754 - val_loss: 0.0146 - val_mae: 0.0460 - lr: 0.0010
Epoch 2/50
1086/1086 [==============================] - 2s 2ms/step - loss: 0.0225 - mae: 0.0573 - val_loss: 0.0135 - val_mae: 0.0454 - lr: 0.0010
Epoch 3/50
1086/1086 [==============================] - 2s 2ms/step - loss: 0.0212 - mae: 0.0540 - val_loss: 0.0141 - val_mae: 0.0449 - lr: 0.0010
Epoch 4/50
1086/1086 [==============================] - 2s 2ms/step - loss: 0.0205 - mae: 0.0533 - val_loss: 0.0205 - val_mae: 0.0570 - lr: 0.0010
Epoch 5

1086/1086 [==============================] - 3s 3ms/step - loss: 0.0209 - mae: 0.0582 - val_loss: 0.0264 - val_mae: 0.0783 - lr: 0.0010
Epoch 6/50
1086/1086 [==============================] - 3s 3ms/step - loss: 0.0204 - mae: 0.0557 - val_loss: 0.0181 - val_mae: 0.0556 - lr: 0.0010
Epoch 7/50
1086/1086 [==============================] - 3s 3ms/step - loss: 0.0201 - mae: 0.0544 - val_loss: 0.0174 - val_mae: 0.0523 - lr: 0.0010
Epoch 8/50
1086/1086 [==============================] - 3s 3ms/step - loss: 0.0200 - mae: 0.0530 - val_loss: 0.0134 - val_mae: 0.0377 - lr: 0.0010
Epoch 9/50
1086/1086 [==============================] - 3s 3ms/step - loss: 0.0197 - mae: 0.0522 - val_loss: 0.0165 - val_mae: 0.0536 - lr: 0.0010
Epoch 10/50
1086/1086 [==============================] - 3s 3ms/step - loss: 0.0194 - mae: 0.0510 - val_loss: 0.0175 - val_mae: 0.0495 - lr: 0.0010
Epoch 11/50
1086/1086 [==============================] - 3s 3ms/step - loss: 0.0195 - mae: 0.0495 - val_loss: 0.0150 - val_mae: 

Epoch 14/50
1086/1086 [==============================] - 2s 2ms/step - loss: 0.0178 - mae: 0.0442 - val_loss: 0.0159 - val_mae: 0.0468 - lr: 1.0000e-04
Epoch 15/50
1086/1086 [==============================] - 2s 2ms/step - loss: 0.0178 - mae: 0.0443 - val_loss: 0.0172 - val_mae: 0.0502 - lr: 1.0000e-04
Epoch 16/50
1086/1086 [==============================] - 2s 2ms/step - loss: 0.0178 - mae: 0.0443 - val_loss: 0.0166 - val_mae: 0.0482 - lr: 1.0000e-04
Epoch 17/50
1086/1086 [==============================] - 2s 2ms/step - loss: 0.0178 - mae: 0.0443 - val_loss: 0.0162 - val_mae: 0.0483 - lr: 1.0000e-04
Epoch 18/50
1086/1086 [==============================] - 2s 2ms/step - loss: 0.0178 - mae: 0.0442 - val_loss: 0.0201 - val_mae: 0.0563 - lr: 1.0000e-04
Epoch 19/50
1086/1086 [==============================] - 2s 2ms/step - loss: 0.0178 - mae: 0.0444 - val_loss: 0.0178 - val_mae: 0.0505 - lr: 1.0000e-04
Epoch 20/50
1086/1086 [==============================] - 2s 2ms/step - loss: 0.0178 - ma

In [145]:
# Train 10 Models for UK - Ghi

N_MODELS = 10
uk_models_tn_ghi = {}

for i in np.arange(N_MODELS):
    IPython.display.clear_output()
    print('Current Model: {:02d}'.format(i+1))
    
    tabnet_regressor = StackedTabNetRegressor(feature_columns=None,
                                              num_layers=1, #study.best_params['num_layers'],
                                              num_regressors=1, 
                                              feature_dim=study_ghi.best_params['feature_dim'],
                                              num_features=1, 
                                              output_dim=study_ghi.best_params['output_dim'],
                                              num_decision_steps=study_ghi.best_params['num_decision_steps'],
                                              num_groups=1)
    current_net_history = compile_and_fit(tabnet_regressor, 
                                          X_train['uk']['irradiance_Wm-2'].values.reshape(-1, 1), y_train['uk'],
                                          X_val['uk']['irradiance_Wm-2'].values.reshape(-1, 1), y_val['uk'])
    
    tabnet_regressor.save('models/tabnet/uk_ghi/Model {:02d}'.format(i+1), save_format='tf')
    uk_models_tn_ghi['Model {:02d}'.format(i+1)] = tabnet_regressor

Current Model: 10
Epoch 1/1000
1086/1086 [==============================] - 2s 1ms/step - loss: 0.0623 - mae: 0.0874 - val_loss: 0.0137 - val_mae: 0.0454 - lr: 0.0010
Epoch 2/1000
1086/1086 [==============================] - 1s 953us/step - loss: 0.0230 - mae: 0.0594 - val_loss: 0.0132 - val_mae: 0.0473 - lr: 0.0010
Epoch 3/1000
1086/1086 [==============================] - 1s 952us/step - loss: 0.0217 - mae: 0.0549 - val_loss: 0.0136 - val_mae: 0.0420 - lr: 0.0010
Epoch 4/1000
1086/1086 [==============================] - 1s 947us/step - loss: 0.0207 - mae: 0.0540 - val_loss: 0.0199 - val_mae: 0.0555 - lr: 0.0010
Epoch 5/1000
1086/1086 [==============================] - 1s 962us/step - loss: 0.0204 - mae: 0.0525 - val_loss: 0.0266 - val_mae: 0.0677 - lr: 0.0010
Epoch 6/1000
1086/1086 [==============================] - 1s 952us/step - loss: 0.0200 - mae: 0.0514 - val_loss: 0.0172 - val_mae: 0.0521 - lr: 0.0010
Epoch 7/1000
1086/1086 [==============================] - 1s 960us/step - loss

Epoch 55/1000
1086/1086 [==============================] - 1s 961us/step - loss: 0.0184 - mae: 0.0455 - val_loss: 0.0199 - val_mae: 0.0575 - lr: 0.0010
Epoch 56/1000
1086/1086 [==============================] - 1s 953us/step - loss: 0.0182 - mae: 0.0455 - val_loss: 0.0221 - val_mae: 0.0598 - lr: 0.0010
Epoch 57/1000
1086/1086 [==============================] - 1s 948us/step - loss: 0.0184 - mae: 0.0459 - val_loss: 0.0163 - val_mae: 0.0476 - lr: 0.0010
Epoch 58/1000
1086/1086 [==============================] - 1s 948us/step - loss: 0.0182 - mae: 0.0454 - val_loss: 0.0169 - val_mae: 0.0502 - lr: 0.0010
Epoch 59/1000
1086/1086 [==============================] - 1s 950us/step - loss: 0.0183 - mae: 0.0454 - val_loss: 0.0163 - val_mae: 0.0493 - lr: 0.0010
Epoch 60/1000
1086/1086 [==============================] - 1s 941us/step - loss: 0.0183 - mae: 0.0455 - val_loss: 0.0144 - val_mae: 0.0431 - lr: 0.0010
Epoch 61/1000
1086/1086 [==============================] - 1s 947us/step - loss: 0.0182 

Epoch 109/1000
1086/1086 [==============================] - 1s 960us/step - loss: 0.0182 - mae: 0.0453 - val_loss: 0.0146 - val_mae: 0.0426 - lr: 0.0010
Epoch 110/1000
1086/1086 [==============================] - 1s 957us/step - loss: 0.0180 - mae: 0.0448 - val_loss: 0.0166 - val_mae: 0.0487 - lr: 0.0010
Epoch 111/1000
1086/1086 [==============================] - 1s 958us/step - loss: 0.0181 - mae: 0.0447 - val_loss: 0.0203 - val_mae: 0.0538 - lr: 0.0010
Epoch 112/1000
1086/1086 [==============================] - 1s 962us/step - loss: 0.0182 - mae: 0.0453 - val_loss: 0.0164 - val_mae: 0.0486 - lr: 0.0010
Epoch 113/1000
1086/1086 [==============================] - 1s 959us/step - loss: 0.0182 - mae: 0.0449 - val_loss: 0.0160 - val_mae: 0.0488 - lr: 0.0010
Epoch 114/1000
1086/1086 [==============================] - 1s 961us/step - loss: 0.0181 - mae: 0.0448 - val_loss: 0.0158 - val_mae: 0.0458 - lr: 0.0010
Epoch 115/1000
1086/1086 [==============================] - 1s 953us/step - loss: 

1086/1086 [==============================] - 1s 976us/step - loss: 0.0177 - mae: 0.0438 - val_loss: 0.0158 - val_mae: 0.0464 - lr: 1.0000e-04
Epoch 163/1000
1086/1086 [==============================] - 1s 985us/step - loss: 0.0177 - mae: 0.0438 - val_loss: 0.0165 - val_mae: 0.0481 - lr: 1.0000e-04
Epoch 164/1000
1086/1086 [==============================] - 1s 973us/step - loss: 0.0177 - mae: 0.0438 - val_loss: 0.0170 - val_mae: 0.0493 - lr: 1.0000e-04
Epoch 165/1000
1086/1086 [==============================] - 1s 959us/step - loss: 0.0177 - mae: 0.0438 - val_loss: 0.0164 - val_mae: 0.0481 - lr: 1.0000e-04
Epoch 166/1000
1086/1086 [==============================] - 1s 952us/step - loss: 0.0177 - mae: 0.0438 - val_loss: 0.0163 - val_mae: 0.0473 - lr: 1.0000e-04
Epoch 167/1000
1086/1086 [==============================] - 1s 952us/step - loss: 0.0177 - mae: 0.0438 - val_loss: 0.0162 - val_mae: 0.0474 - lr: 1.0000e-04
Epoch 168/1000
1086/1086 [==============================] - 1s 961us/step

1086/1086 [==============================] - 1s 955us/step - loss: 0.0177 - mae: 0.0438 - val_loss: 0.0180 - val_mae: 0.0516 - lr: 1.0000e-04
Epoch 215/1000
1086/1086 [==============================] - 1s 957us/step - loss: 0.0177 - mae: 0.0437 - val_loss: 0.0171 - val_mae: 0.0494 - lr: 1.0000e-04
Epoch 216/1000
1086/1086 [==============================] - 1s 946us/step - loss: 0.0177 - mae: 0.0439 - val_loss: 0.0166 - val_mae: 0.0485 - lr: 1.0000e-04
Epoch 217/1000
1086/1086 [==============================] - 1s 928us/step - loss: 0.0177 - mae: 0.0438 - val_loss: 0.0165 - val_mae: 0.0479 - lr: 1.0000e-04
Epoch 218/1000
1086/1086 [==============================] - 1s 933us/step - loss: 0.0177 - mae: 0.0438 - val_loss: 0.0164 - val_mae: 0.0479 - lr: 1.0000e-04
Epoch 219/1000
1086/1086 [==============================] - 1s 932us/step - loss: 0.0177 - mae: 0.0438 - val_loss: 0.0166 - val_mae: 0.0481 - lr: 1.0000e-04
Epoch 220/1000
1086/1086 [==============================] - 1s 931us/step

In [146]:
# Aux Function for predicting and storing values

def do_predictions(dictionary, save_path, X, y, index):
    # Go through each model in the dictionary
    for model in dictionary.keys():
        print('Doing {}'.format(model))
        
        temp_path = '{}/{}.csv'.format(save_path, model)
        
        y_pred = dictionary[model].predict(X, verbose=1)
        y_pred = pd.DataFrame(y_pred[:, :], columns=['pv_power_mw'],
                              index=index)
        
        y_pred.to_csv(temp_path)
        
    # Also save ground-truth data at the end of the loop
    y_true = pd.DataFrame(y, columns=['pv_power_mw'],
                          index=index)
    
    temp_path_gt = '{}/gt.csv'.format(save_path)
    y_true.to_csv(temp_path_gt)
    
    
def predict_upacs_ghi(model_dictionary, upac_name, X_train, y_train, X_val, y_val, X_test, y_test):
    # Simply call the function above for each of the settings to simplify
    
    print('Doing training for {}'.format(upac_name))
    temp_path = 'results/tabnet/{}_ghi/train'.format(upac_name)
    do_predictions(dictionary=model_dictionary, 
                   save_path=temp_path, 
                   X=X_train[upac_name]['irradiance_Wm-2'].values.reshape(-1, 1), 
                   y=y_train[upac_name],
                   index=normalized_train[upac_name].index)
    IPython.display.clear_output()
    
    print('Doing validation for {}'.format(upac_name))
    temp_path = 'results/tabnet/{}_ghi/val'.format(upac_name)
    do_predictions(dictionary=model_dictionary, 
                   save_path=temp_path, 
                   X=X_val[upac_name]['irradiance_Wm-2'].values.reshape(-1, 1), 
                   y=y_val[upac_name],
                   index=normalized_val[upac_name].index)
    IPython.display.clear_output()
    
    print('Doing testing for {}'.format(upac_name))
    temp_path = 'results/tabnet/{}_ghi/test'.format(upac_name)
    do_predictions(dictionary=model_dictionary, 
                   save_path=temp_path, 
                   X=X_test[upac_name]['irradiance_Wm-2'].values.reshape(-1, 1), 
                   y=y_test[upac_name],
                   index=normalized_test[upac_name].index)
    IPython.display.clear_output()

In [147]:
# UPAC08 Predictions for all features

predict_upacs_ghi(model_dictionary=uk_models_tn_ghi, 
                  upac_name='uk',
                  X_train=X_train, y_train=y_train,
                  X_val=X_val, y_val=y_val,
                  X_test=X_test, y_test=y_test)